# Update RDS file of Negative References

**Note 1:** You need to get ``old_rds_sizes_path`` file from server beforehand (includes calculated genome sizes after cleaning) and adjust the filepaths below. If genbank is already downloaded, set ``genbank_exists`` to ``True``.

**Note 2:** This notebook will be moved to ``prepare_input.py`` in an adapted version.

In [1]:
genbank_exists = True

genbank_path = '../data/simulation/assembly_summary_genbank_281120222.txt'
old_rds_sizes_path = '../data/simulation/metadata_neg_ref_sizes.rds'  #  from /hpi/fs00/share/fg/renard/nina.ihde/data/simulation/metadata_neg_ref_sizes.rds
old_rds_path = '../data/simulation/IMG_1_folds_170418_sizes.rds'
new_rds_path = '../data/simulation/IMG_1_folds_170418_sizes_updated.rds'

In [2]:
import pyreadr

size_rds = pyreadr.read_r(old_rds_sizes_path)[None]
size_rds.head()

,bioproject,assembly_accession,taxid,species_taxid,organism_name,organism_name.gb,infraspecific_name,version_status,assembly_level,seq_rel_date,...,IMG.Genome.ID,NCBI.Bioproject.Accession,Species,Strain,Genome.Name...Sample.Name,Add.Date,Pathogenic,fold1,subset,Genome.Size
rownames,,,,,,,,,,,,,,,,,,,,,
1,PRJNA16421,GCA_000009245.1,376619,263,Francisella tularensis subsp. holarctica LVS,Francisella tularensis subsp. holarctica LVS,strain=LVS,latest,Complete Genome,2006-03-02,...,6.370001e+08,PRJNA16421,Francisella tularensis,LVS,Francisella tularensis holarctica LVS,2006-10-02,False,train,selected,1895994.0
2,PRJNA62415,GCA_000248335.2,981326,48296,Acinetobacter pittii DSM 21653,Acinetobacter pittii DSM 21653,strain=DSM 21653,latest,Contig,2012-02-21,...,2.547132e+09,PRJNA62415,Acinetobacter pittii,DSM 21653,"Acinetobacter genomospecies 3, DSM 21653",2013-11-27,False,train,selected,3746320.0
3,PRJNA81141,GCA_000252915.3,1134455,571,Klebsiella oxytoca 11492-1,Klebsiella oxytoca 11492-1,strain=11492-1,latest,Contig,2012-03-16,...,2.548877e+09,PRJNA81141,Klebsiella oxytoca,11492-1,Klebsiella oxytoca 11492-1,2013-12-05,False,train,selected,6166501.0
4,PRJNA68097,GCA_000219275.1,1046625,28090,Acinetobacter lwoffii WJ10621,Acinetobacter lwoffii WJ10621,strain=WJ10621,latest,Chromosome,2011-06-27,...,2.547132e+09,PRJNA68097,Acinetobacter lwoffii,WJ10621,Acinetobacter lwoffii WJ10621,2013-11-27,False,train,selected,3184320.0
5,PRJNA46457,GCA_000875755.1,743967,51365,Mycoplasma yeatsii GM274B,Mycoplasma yeatsii GM274B,strain=GM274B,latest,Complete Genome,2015-02-13,...,2.630968e+09,PRJNA46457,Mycoplasma yeatsii,GM274B,Mycoplasma yeatsii GM274B,2015-09-21,False,train,selected,895051.0


In [6]:
import numpy as np

# extract non-found species
df = size_rds[size_rds['Genome.Size'] == 0][['Species', 'ftp_path']].reset_index(drop=True).rename(columns={'ftp_path': 'Current Path'})
df['Updated Path'] = np.nan
df

,Species,Current Path,Updated Path
0,Streptococcus suis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
1,Mycoplasma hyorhinis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
2,Burkholderia cenocepacia,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
3,Serratia proteamaculans,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
4,Ralstonia pickettii,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
5,Histophilus somni,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
6,Yersinia pseudotuberculosis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
7,Shewanella putrefaciens,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
8,Thermobifida fusca,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
9,Acinetobacter nosocomialis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN


In [7]:
# path for contig "ASM21129v1" can stay as it is, because it was falsely removed (was compared to no real plasmid)
# NOTE: this manual path setting won't be needed in prepare_input.py as this script will be applied before cleaning
df.loc[df['Species'] == 'Mycoplasma hyorhinis', ['Updated Path']] = df[df['Species'] == 'Mycoplasma hyorhinis']['Current Path']
df

,Species,Current Path,Updated Path
0,Streptococcus suis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
1,Mycoplasma hyorhinis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
2,Burkholderia cenocepacia,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
3,Serratia proteamaculans,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
4,Ralstonia pickettii,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
5,Histophilus somni,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
6,Yersinia pseudotuberculosis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
7,Shewanella putrefaciens,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
8,Thermobifida fusca,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN
9,Acinetobacter nosocomialis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN


In [23]:
import wget

if not genbank_exists:
    wget.download('https://ftp.ncbi.nlm.nih.gov/genomes/genbank/assembly_summary_genbank.txt',
                  out=genbank_path)

In [10]:
import pandas as pd

genbank = pd.read_csv(genbank_path, sep='\t', skiprows=[0])
genbank.rename(columns={'# assembly_accession': 'assembly_accession'}, inplace=True)
genbank.head()

,assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date
0,GCA_000001215.4,PRJNA13812,SAMN02803731,NaN,reference genome,7227,7227,Drosophila melanogaster,NaN,NaN,...,Full,2014/08/01,Release 6 plus ISO1 MT,The FlyBase Consortium/Berkeley Drosophila Gen...,GCF_000001215.4,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,NaN,NaN,na
1,GCA_000001405.29,PRJNA31257,NaN,NaN,reference genome,9606,9606,Homo sapiens,NaN,NaN,...,Full,2022/02/03,GRCh38.p14,Genome Reference Consortium,GCF_000001405.40,different,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,NaN,NaN,na
2,GCA_000001515.5,PRJNA13184,SAMN02981217,AACZ00000000.4,na,9598,9598,Pan troglodytes,NaN,Yerkes chimp pedigree #C0471 (Clint),...,Full,2016/05/03,Pan_tro 3.0,Chimpanzee Sequencing and Analysis Consortium,GCF_000001515.7,different,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,superseded by newer assembly for species,NaN,na
3,GCA_000001545.3,PRJNA20869,SAMN02981238,ABGA00000000.1,na,9601,9601,Pongo abelii,NaN,ISIS 71,...,Full,2008/11/13,P_pygmaeus_2.0.2,Orangutan Genome Sequencing Consortium,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,NaN,NaN,na
4,GCA_000001635.9,PRJNA20689,NaN,NaN,reference genome,10090,10090,Mus musculus,strain=C57BL/6J,NaN,...,Full,2020/06/24,GRCm39,Genome Reference Consortium,GCF_000001635.27,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,NaN,NaN,na


In [11]:
def get_completeness_nr(assembly_level):
    if assembly_level == 'Complete Genome': return 4
    elif assembly_level == 'Chromosome': return 3
    elif assembly_level == 'Scaffold': return 2
    elif assembly_level == 'Contig': return 1
    else: raise ValueError(f'Non-valid assembly level: {assembly_level}!')

In [12]:
from numpy import random

random_gen = random.default_rng(42)

for species in df[df['Updated Path'].isna()]['Species']:
    print(f'Current species: {species}')
    species_genomes = genbank[genbank['organism_name'] == species].reset_index(drop=True)

    if not species_genomes[species_genomes['refseq_category'] != 'na'].empty:
        print('Representative exists\n')
        df.loc[df['Species'] == species, ['Updated Path']] = species_genomes[species_genomes['refseq_category'] != 'na'].iloc[0]['ftp_path']
    else:
        print('Representative does not exist!\n')
        # add numerical genome completeness
        species_genomes['completeness_nr'] = species_genomes['assembly_level'].apply(lambda l: get_completeness_nr(l))
        # get most complete ones
        most_complete = species_genomes[species_genomes['completeness_nr'] == species_genomes['completeness_nr'].max()]
        # select randomly among most complete ones
        df.loc[df['Species'] == species, ['Updated Path']] = most_complete.sample(n=1, random_state=random_gen).iloc[0]['ftp_path']

Current species: Streptococcus suis
Representative does not exist!

Current species: Burkholderia cenocepacia
Representative exists

Current species: Serratia proteamaculans
Representative exists

Current species: Ralstonia pickettii
Representative exists

Current species: Yersinia pseudotuberculosis
Representative does not exist!

Current species: Shewanella putrefaciens
Representative does not exist!

Current species: Thermobifida fusca
Representative exists

Current species: Acinetobacter nosocomialis
Representative does not exist!



In [13]:
df

,Species,Current Path,Updated Path
0,Streptococcus suis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...
1,Mycoplasma hyorhinis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
2,Burkholderia cenocepacia,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...
3,Serratia proteamaculans,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...
4,Ralstonia pickettii,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...
5,Histophilus somni,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...
6,Yersinia pseudotuberculosis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...
7,Shewanella putrefaciens,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...
8,Thermobifida fusca,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...
9,Acinetobacter nosocomialis,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...


In [14]:
import pyreadr

# load current RDS
rds = pyreadr.read_r(old_rds_path)[None]
rds['ftp_path'] = rds['ftp_path'].astype(str)

# iterate new paths of non-found species
for species, updated_path in zip(df['Species'], df['Updated Path']):
    if updated_path != rds[rds['Species'] == species].iloc[0]['ftp_path']:
        # update current RDS, keep ftp-prefix for consistency
        print(updated_path)
        rds.loc[rds['Species'] == species, ['ftp_path']] = updated_path.replace('https://', 'ftp://')
    else:
        print(f'Path for {species} is still the same')

# save updated RDS
pyreadr.write_rds(new_rds_path, rds)

https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/901/543/625/GCA_901543625.1_42685_E01
Path for Mycoplasma hyorhinis is still the same
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/718/895/GCA_001718895.1_ASM171889v1
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/008/830/365/GCA_008830365.1_ASM883036v1
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/374/465/GCA_902374465.1_MGYG-HGUT-01384
ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/019/405/GCF_000019405.1_ASM1940v1
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/750/315/GCA_000750315.1_ASM75031v1
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/025/402/875/GCA_025402875.1_ASM2540287v1
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/015/034/585/GCA_015034585.1_ASM1503458v1
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/814/165/GCA_000814165.3_ASM81416v3
